In [1]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import *
import tensorflow as tf

from _context import dl_uncertainty
import dl_uncertainty.tf_utils as tf_utils
import dl_uncertainty.tf_utils.layers as layers

In [2]:
x = tf.constant(np.array([[1,2],[3,4.0]]), dtype=tf.float32)
sess = tf.Session()

## 1. Sampling random operations with no variables

### 1.1. Dropout

In [3]:
@layers.scoped
def dropout(x):
    return tf.nn.dropout(x, keep_prob=0.5)

ys = layers.sample(lambda x: dropout(x, name='do'), x, 100)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x
ys2 = layers.sample(lambda x: dropout(x, name='do'), x, 8, examplewise=True)

asdasfasfasfasdasdsa 


NameError: name 'exit' is not defined

In [ ]:
sess.run(tf.global_variables_initializer())

print('x')
print(sess.run(x))

print('')
print('dropout(x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys)
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean)
print(m)

print('')
print('Example-wise dropout(x) sampling (first 2 elements)')
s = sess.run(ys2)
print(s[:2], '...')
print('shape:', s.shape)

### 1.2. `tf.random_*`, `use_tf_loop=False`

In [ ]:
@layers.scoped
def random(m, sd):
    print(x.shape)
    return tf.random_normal(shape=m.shape, mean=m, stddev=sd)

ys = layers.sample(lambda x: random(x, x), x, 100, examplewise=False, use_tf_loop=False)
y_mean = tf.reduce_mean(ys, 0)  # should approximate x

In [4]:
print('')
print('random_normal(x, x) sampling ([n,N,...]) (first 3 samples)')
s = sess.run(ys)
print(s[:3], '...')
print('shape:', s.shape)

print('')
print('Sample mean (estimation of x)')
m = sess.run(y_mean)
print(m)


random_normal(x, x) sampling ([n,N,...]) (first 3 samples)


NameError: name 'ys' is not defined

## 2. Sampling operations with variables

In [10]:
def init_func(*args, **kwargs): return tf.random_uniform([])

@layers.scoped
def rando(x):
    v = tf.get_variable("var", initializer=tf.random_uniform(x.shape))
    #tf.get_variable_scope().reuse_variables()
    print(v)
    return tf.random_uniform(x.shape)*v*tf.identity(x)

In [11]:
#trando = tf.make_template('rando', rando)
#rr = [tf.map_fn(rando, x) for _ in range(2)]
samp = layers.sample(rando, x, 100, use_tf_loop=False)

<tf.Variable 'sample_2/rando/var:0' shape=() dtype=float32_ref>


In [12]:
sess.run(tf.global_variables_initializer())

#rr_ = sess.run(rr)
samp_ = sess.run(samp)
print(samp_)

#for a in rr_:
#    print(a)

[[[1.89889350e-03 1.23591803e-01]
  [1.91727206e-01 5.53176880e-01]]

 [[1.21515438e-01 1.53365061e-01]
  [2.98762590e-01 5.19324720e-01]]

 [[3.65770683e-02 2.68574029e-01]
  [2.16931663e-02 5.64989567e-01]]

 [[1.03111984e-02 2.78570950e-01]
  [1.17893711e-01 1.05087578e-01]]

 [[3.99599038e-02 8.19527283e-02]
  [2.74180844e-02 1.70953140e-01]]

 [[7.58823678e-02 2.02116206e-01]
  [8.91020298e-02 4.59489465e-01]]

 [[6.60599992e-02 4.58325632e-02]
  [4.16928768e-01 5.54927766e-01]]

 [[9.33806747e-02 2.03214571e-01]
  [3.72164994e-01 3.23068053e-02]]

 [[9.67135727e-02 1.66488603e-01]
  [8.60765725e-02 2.60945916e-01]]

 [[1.01071849e-01 6.51322827e-02]
  [2.99444556e-01 4.97165173e-01]]

 [[6.76757172e-02 2.04102710e-01]
  [7.45904222e-02 3.90253246e-01]]

 [[5.10625914e-03 7.77763724e-02]
  [4.03987288e-01 3.95837426e-01]]

 [[6.26661405e-02 1.06704514e-03]
  [3.53792787e-01 3.21665615e-01]]

 [[9.37907845e-02 1.70786321e-01]
  [2.45273173e-01 3.83480638e-01]]

 [[5.67786396e-02 2.

In [8]:
# samples should be different because outputs come from different subgraphs with different values of "var"
assert np.any(rr_[0] != rr_[1])

NameError: name 'rr_' is not defined

In [ ]:
samp_ = sess.run(samp)
print(samp_)

In [ ]:
# samples should be equal because outputs come from the same subgraph with a single "var"
assert np.all(samp_[0]==samp_[1])  # samples should be different

In [ ]:
v = tf.get_variable("var", initializer=tf.random_uniform(shape=tf.shape(x)))

@layers.scoped
def wrapv(fn, x):
    with tf.control_dependencies([tf.assign(v, x)]):
        return fn(v)
samp = layers.sample(lambda x: wrapv(rando, x), x, 10)

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(samp)